In [56]:
from glmhmm import *
import numpy as np
import sys
print(sys.path)
# sys.path.append('/data/users/weixuan/work/model/other_folder')
# from my_module import my_function  # Replace with your actual module and function names


['/data/users/weixuan/work/model/glmhmm', '/home/wliu25/miniconda3/envs/glmhmm/lib/python38.zip', '/home/wliu25/miniconda3/envs/glmhmm/lib/python3.8', '/home/wliu25/miniconda3/envs/glmhmm/lib/python3.8/lib-dynload', '', '/home/wliu25/.local/lib/python3.8/site-packages', '/data/users/weixuan/work/model/ssm', '/home/wliu25/miniconda3/envs/glmhmm/lib/python3.8/site-packages', '/home/wliu25/miniconda3/envs/glmhmm/lib/python3.8/site-packages/setuptools/_vendor']


In [3]:
num_data = 500
model_true = GLMHMM(num_data, 2, 1, 1) #N, n_states, n_features, n_outputs
model_true.transition_matrix = np.array([[0.5, 0.5], [0.5, 0.5]])

In [4]:
X, Y, states = model_true.generate_data(num_data)

In [6]:
init = 3
lls_all = np.zeros((init, 250))
A_all = np.zeros((init, 2, 2))
w_all = np.zeros((init, 2, 2, 1))
pi0_all = np.zeros((init, 2))
for i in range(init):
  print(f'-----------------init {i}-----------------')
  model_pred = GLMHMM(num_data, 2, 1, 1)
  A=model_pred.transition_matrix
  w=model_pred.w
  lls,A,w,pi0 = model_pred.fit(Y,X,A,w, fit_init_states=True)
  lls_all[i] = lls
  A_all[i] = A
  w_all[i] = w
  pi0_all[i] = pi0

-----------------init 0-----------------
equals 1


/data/users/weixuan/work/model/glmhmm/glmhmm.py:194: RuntimeWarning: invalid value encountered in divide
  alpha[i] = pxz/cs[i] # conditional p(z_t | y_1:t)
/data/users/weixuan/work/model/glmhmm/glmhmm.py:196: RuntimeWarning: divide by zero encountered in log
  ll = np.sum(np.log(cs))
/data/users/weixuan/work/model/glmhmm/glmhmm.py:290: RuntimeWarning: divide by zero encountered in log
  ll_list = [gammak[i] * np.log(self.dist_pdf(y[i], thetak[i], otherparamk=otherparamk)) for i in range(self.N)]


equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
equals 1
e

In [ ]:
hmm = ssm.HMM(2, 1, M=1, transitions="inputdriven")
hmm_lls = hmm.fit(Y, inputs=X, method="em", num_iters=100, init_method="kmeans")

In [ ]:
hmm.transitions.Ws, np.exp(hmm.transitions.log_Ps)

In [ ]:
hmm = ssm.HMM(2, 1)
hmm_lls = hmm.fit(Y, method="em", num_iters=100, init_method="kmeans")
hmm.transitions.transition_matrix

In [30]:
def find_best_fit(lls):
    cleaned_lls = lls[~np.isnan(lls).all(axis=1)]
    assert len(cleaned_lls) != 0
    return np.argmax(np.nanmax(cleaned_lls,axis=1))

In [31]:
bestidx = find_best_fit(lls_all) # run several times (several inti) and get the best one
A_pred = A_all[bestidx]
w_pred = w_all[bestidx]
pi0_pred = pi0_all[bestidx]

AssertionError: 

In [ ]:
model_true = GLMHMM(num_data, 2, 1, 1) #N, n_states, n_features, n_outputs
model_true.transition_matrix = np.array([[0.5, 0.5], [0.5, 0.5]])

In [ ]:
#N = 500, 1000, n_state = 3, n_features(x_d)=?4?7? same as agent, n_output(dim(y))=2(same as agent)
# w is of shape self.n_states, self.n_features + 1, self.n_outputs
# w in R^{n_features + 1 x n_states}

# Easy Level (2 hidden states)

In [51]:
N=500
K=2
D=2
dim_output=2

In [57]:
div_pt = np.linspace(-1, 1, K+1)
w_true = np.zeros((K, D, dim_output))
for d in range(D):
    #k=0 -> neg
    #k=1 -> pos
    for k in range(K):
        w_true[k, d, :] = np.random.uniform(low=div_pt[k], high=div_pt[k+1], \
                size = dim_output)


w_true *= 2
w_true = np.pad(w_true, ((0, 0), (0, 1), (0, 0)), mode='constant')
w_true

array([[[-1.60791253, -0.47297595],
        [-0.62221878, -1.78284146],
        [ 0.        ,  0.        ]],

       [[ 1.03599355,  1.62867211],
        [ 1.2345478 ,  1.69767091],
        [ 0.        ,  0.        ]]])

In [58]:
model_true = GLMHMM(N, K, D, dim_output) #N, n_states, n_features, n_outputs
model_true.transition_matrix = np.array([[0.5, 0.5], [0.5, 0.5]])
model_true.w = w_true
X, Y, states = model_true.generate_data(num_data)

In [59]:
model_pred = GLMHMM(N, K, D, dim_output)
A=model_pred.transition_matrix
w=model_pred.w
lls,A,w,pi0 = model_pred.fit(Y,X,A,w, fit_init_states=True)

/data/users/weixuan/work/model/glmhmm/glmhmm.py:186: RuntimeWarning: invalid value encountered in divide
  
/data/users/weixuan/work/model/glmhmm/glmhmm.py:196: RuntimeWarning: divide by zero encountered in log
  alpha_prior[i] = alpha[i-1]@A # propogate uncertainty forward
/data/users/weixuan/work/model/glmhmm/glmhmm.py:290: RuntimeWarning: divide by zero encountered in log
  """


equals 1
equals 1


KeyboardInterrupt: 

In [38]:
A

array([[nan, nan],
       [nan, nan]])

In [39]:
w_true, w

(array([[[-1.64742791, -0.3887375 ],
         [-1.9785354 , -0.49867193],
         [ 0.        ,  0.        ]],
 
        [[ 1.5036048 ,  0.3686037 ],
         [ 1.85819533,  0.23800501],
         [ 0.        ,  0.        ]]]),
 array([[[ 1.84262533, -0.42605041],
         [-1.87340022,  0.24775204],
         [ 0.52969823, -0.64436564]],
 
        [[ 0.12106484,  0.92819894],
         [ 0.62714374, -0.47769858],
         [ 0.79717817, -0.09500238]]]))

In [ ]:
N=500
K=3
D=4
dim_output=2



In [ ]:
for k in 